In [7]:
import os
import subprocess
from collections import defaultdict
import cv2
from realesrgan import RealESRGANer
from PIL import Image, ImageEnhance, ImageFilter
from torchvision.transforms import transforms
from torchvision import transforms
from basicsr.archs.swinir_arch import SwinIR
from basicsr.archs.rrdbnet_arch import RRDBNet
from pathlib import Path
import torch
from pathlib import Path
import numpy as np
import re
import sys

In [8]:
os.environ["PATH"] = r"C:\ffmpeg\ffmpeg-7.1-full_build\bin" + ";" + os.environ["PATH"]
# rife_path = r"D:\oldMemories\RIFE"
# sys.path.append(rife_path)

In [9]:
# Define the source folder to traverse
SOURCE_FOLDER = r"F:\consolidated old photos"

# Define the output folder for converted videos (change if needed)
OUTPUT_FOLDER = r"F:\consolidated old photos\converted_videos"

# Test data
TEST_SOURCE_FOLDER = r"D:\oldMemoriesTestArea\original"
TEST_OUTPUT_FOLDER = r"D:\oldMemoriesTestArea\converted_videos"
TEST_EXTRACTS_FOLDER = r"D:\oldMemoriesTestArea\extracts"
TEST_SWINIR_FOLDER = r"D:\oldMemoriesTestArea\swinir"
TEST_UPSCALED_FOLDER = r"D:\oldMemoriesTestArea\upscaled"
FINAL_VIDEO_FOLDER = r"D:\oldMemoriesTestArea\final_videos"

# real esrgan model
REALESRGAN_MODEL_PATH = r"D:\oldMemories\RealESRGAN_x4plus.pth"
SWINIR_MODEL_PATH = r"D:\oldMemories\005_colorDN_DFWB_s128w8_SwinIR-M_noise15.pth"

RIFE_SCRIPT = r"D:\oldMemories\RIFE\inference_video.py"

# Video extensions that need standardizing
TARGET_EXTENSIONS = {".avi", ".dat", ".mpg", ".vob"}

# Info on the video files

In [ ]:
def get_video_formats_and_paths(folder_path):
    """
    Traverse all subfolders, count video file formats, and list paths for each format.

    Args:
    - folder_path (str): Path to the folder to scan.

    Returns:
    - Dict with formats as keys, and a tuple (count, list of paths) as values.
    """
    # Define common video file extensions
    video_extensions = {
        ".mp4", ".avi", ".mkv", ".mov", ".wmv", ".flv", ".webm", ".mpeg", ".3gp", ".mpg", ".m4v", ".vob", ".dat"
    }

    # Dictionary to store format counts and paths
    format_data = defaultdict(lambda: {"count": 0, "paths": []})

    # Walk through the folder and its subfolders
    for root, _, files in os.walk(folder_path):
        for file in files:
            _, ext = os.path.splitext(file)
            ext = ext.lower()
            if ext in video_extensions:
                full_path = os.path.join(root, file)
                format_data[ext]["count"] += 1
                format_data[ext]["paths"].append(full_path)

    return dict(format_data)

# Example usage
checkVideoFormatsHere = r"F:\consolidated old photos"  # Replace with your folder path
video_format_data = get_video_formats_and_paths(checkVideoFormatsHere)

# Print results
print(f"Video formats and their counts/paths in '{checkVideoFormatsHere}':\n")
for format, data in sorted(video_format_data.items()):
    print(f"Format: {format}")
    print(f"Count: {data['count']}")
    # if count < 5 print their paths else print only the count
    if data['count'] < 5:
        print("Paths:")
        for path in data["paths"]:
            print(f"  {path}")
    print()


# Standardization of video files

In [10]:
def standardize_videos(source_folder, output_folder):
    """
    Traverse all subfolders from source_folder, find target video formats,
    and convert them to MP4 (H.264 + AAC) in output_folder, mirroring the folder structure.
    """
    for root, _, files in os.walk(source_folder):
        for file_name in files:
            # Get file extension in lowercase
            _, ext = os.path.splitext(file_name)
            ext = ext.lower()

            # If it's one of the target video formats, convert it
            if ext in TARGET_EXTENSIONS:
                source_file_path = os.path.join(root, file_name)

                # Build the mirrored output path by replacing SOURCE_FOLDER with OUTPUT_FOLDER
                relative_path = os.path.relpath(root, source_folder)
                output_subfolder = os.path.join(output_folder, relative_path)
                os.makedirs(output_subfolder, exist_ok=True)

                # Construct output filename (e.g., "Video 4_converted.mp4")
                base_name = os.path.splitext(file_name)[0]
                output_file_path = os.path.join(output_subfolder, base_name + ".mp4")

                print(f"\nConverting: {source_file_path}\n   to --> {output_file_path}")

                # ffmpeg command to convert video to H.264 (libx264) and AAC
                command = [
                    "ffmpeg",
                    "-y",
                    "-i", source_file_path,
                    "-c:v", "libx264",
                    "-crf", "15",        # Lower = higher quality, bigger file
                    "-preset", "slow",   # Or 'medium' / 'slower' / 'veryslow'
                    "-c:a", "aac",       # Encode audio with AAC
                    "-b:a", "192k",      # Optional: set audio bitrate
                    "-movflags", "+faststart",
                    output_file_path
                ]

                try:
                    subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                    print("Conversion successful.")
                except subprocess.CalledProcessError as e:
                    print(f"Error converting {source_file_path}.\nFFmpeg error: {e.stderr.decode('utf-8', errors='replace')}")

if __name__ == "__main__":
    # Make sure output folder exists
    os.makedirs(TEST_OUTPUT_FOLDER, exist_ok=True)

    standardize_videos(TEST_SOURCE_FOLDER, TEST_OUTPUT_FOLDER)
    print("\nAll possible videos have been processed.")


Converting: D:\oldMemoriesTestArea\original\AVSEQ01.DAT
   to --> D:\oldMemoriesTestArea\converted_videos\.\AVSEQ01.mp4
Conversion successful.

Converting: D:\oldMemoriesTestArea\original\flying car.avi
   to --> D:\oldMemoriesTestArea\converted_videos\.\flying car.mp4
Conversion successful.

Converting: D:\oldMemoriesTestArea\original\M2U00418.MPG
   to --> D:\oldMemoriesTestArea\converted_videos\.\M2U00418.mp4
Conversion successful.

Converting: D:\oldMemoriesTestArea\original\VTS_01_3.VOB
   to --> D:\oldMemoriesTestArea\converted_videos\.\VTS_01_3.mp4
Conversion successful.

All possible videos have been processed.


In [11]:
def extract_audio_ffmpeg(input_video_path, output_audio_path):
    """
    Extracts the audio track from a video using FFmpeg without re-encoding the audio.
    -i <input_video>  : Input video file
    -vn               : Disable video
    -acodec copy      : Copy the existing audio track directly
    
    Args:
        input_video_path  (str): Path to the input video (e.g., .mp4)
        output_audio_path (str): Path to save the output audio file (e.g., .aac or .mp3)
    """
    command = [
        "ffmpeg",
        "-y",               # Overwrite output if it exists
        "-i", input_video_path,
        "-vn",             # No video
        "-acodec", "copy", # Copy original audio track without re-encoding
        output_audio_path
    ]
    
    try:
        subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Audio extracted to: {output_audio_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error extracting audio from {input_video_path}.\nFFmpeg error: {e.stderr.decode('utf-8', errors='replace')}")

In [12]:
def extract_frames_opencv(input_video_path, frames_output_folder):
    """
    Extracts frames from a video using OpenCV and saves them as PNG files.
    Args:
        input_video_path     (str): Path to the input .mp4 (or other) video.
        frames_output_folder (str): Folder to save extracted frames.
    """
    os.makedirs(frames_output_folder, exist_ok=True)

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {input_video_path}")
        return

    frame_count = 0
    while True:
        success, frame = cap.read()
        if not success:
            break
        frame_filename = os.path.join(frames_output_folder, f"frame_{frame_count:06d}.png")
        cv2.imwrite(frame_filename, frame)
        frame_count += 1

    cap.release()
    print(f"Extracted {frame_count} frames from {input_video_path} to {frames_output_folder}")

In [13]:
def process_standardized_videos(input_folder, output_folder):
    """
    Traverse all subfolders in `input_folder` to find .mp4 files.
    For each video, extract audio + frames in the corresponding subfolder of `output_folder`.
    
    Args:
        input_folder  (str): Path with standardized videos (likely .mp4).
        output_folder (str): Path to store extracted audio and frames.
    """
    for root, dirs, files in os.walk(input_folder):
        for file_name in files:
            if file_name.lower().endswith(".mp4"):
                video_path = os.path.join(root, file_name)

                # Mirror subfolder structure under output_folder
                relative_path = os.path.relpath(root, input_folder)
                target_subfolder = os.path.join(output_folder, relative_path)
                os.makedirs(target_subfolder, exist_ok=True)

                # Derive base name (no extension)
                base_name = os.path.splitext(file_name)[0]

                # 1) Extract Audio
                # We'll name the audio file base_name.aac in this example
                audio_file_path = os.path.join(target_subfolder, base_name + ".aac")
                extract_audio_ffmpeg(video_path, audio_file_path)

                # 2) Extract Frames
                frames_folder = os.path.join(target_subfolder, f"{base_name}_frames")
                extract_frames_opencv(video_path, frames_folder)

    print("All videos have been processed!")

In [14]:
# Make sure output folder exists
os.makedirs(TEST_EXTRACTS_FOLDER, exist_ok=True)
process_standardized_videos(TEST_OUTPUT_FOLDER, TEST_EXTRACTS_FOLDER)

Audio extracted to: D:\oldMemoriesTestArea\extracts\.\AVSEQ01.aac
Extracted 11196 frames from D:\oldMemoriesTestArea\converted_videos\AVSEQ01.mp4 to D:\oldMemoriesTestArea\extracts\.\AVSEQ01_frames
Audio extracted to: D:\oldMemoriesTestArea\extracts\.\flying car.aac
Extracted 197 frames from D:\oldMemoriesTestArea\converted_videos\flying car.mp4 to D:\oldMemoriesTestArea\extracts\.\flying car_frames
Audio extracted to: D:\oldMemoriesTestArea\extracts\.\M2U00418.aac
Extracted 336 frames from D:\oldMemoriesTestArea\converted_videos\M2U00418.mp4 to D:\oldMemoriesTestArea\extracts\.\M2U00418_frames
Audio extracted to: D:\oldMemoriesTestArea\extracts\.\VTS_01_3.aac
Extracted 24954 frames from D:\oldMemoriesTestArea\converted_videos\VTS_01_3.mp4 to D:\oldMemoriesTestArea\extracts\.\VTS_01_3_frames
All videos have been processed!


# Frame Processing

## Denoising and sharpening

In [15]:
# Load the SwinIR model for denoising
def load_swinir_model(model_path, device):
    model = SwinIR(
        upscale=1,
        in_chans=3,
        img_size=128,
        window_size=8,
        img_range=1.0,
        depths=[6, 6, 6, 6, 6, 6],
        embed_dim=180,
        num_heads=[6, 6, 6, 6, 6, 6],
        mlp_ratio=2,
        upsampler='',
        resi_connection='1conv'
    )
    
    # Load weights
    pretrained = torch.load(model_path, map_location=device)
    param_key_g = 'params' if 'params' in pretrained else list(pretrained.keys())[0]  # Handle different model keys
    model.load_state_dict(pretrained[param_key_g], strict=True)

    model.eval()
    model.to(device)
    return model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = load_swinir_model(SWINIR_MODEL_PATH, device)

c:\Users\Manan\anaconda3\envs\old_mems\Lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
C:\Users\Manan\AppData\Local\Temp\ipykernel_27576\2571553683.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unl

In [16]:
def denoise_image_swinir(model, image_path, device):
    """ Denoise an image using SwinIR and return the output image. """
    # Load and convert image
    img = cv2.imread(str(image_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0  # Normalize to [0,1]

    # Pad image to be a multiple of window size (8)
    h, w, c = img.shape
    h_pad = (8 - h % 8) % 8
    w_pad = (8 - w % 8) % 8
    img = np.pad(img, ((0, h_pad), (0, w_pad), (0, 0)), 'reflect')

    # Convert to tensor
    img_tensor = torch.from_numpy(np.transpose(img, (2, 0, 1))).float().unsqueeze(0).to(device)

    # Run SwinIR model
    with torch.no_grad():
        output = model(img_tensor).clamp_(0, 1)  # Clamp values to valid range

    # Convert back to image
    output = output.squeeze().cpu().numpy().transpose(1, 2, 0)  # CHW → HWC
    output = (output[:h, :w] * 255.0).astype(np.uint8)  # Remove padding & scale back to [0,255]

    return cv2.cvtColor(output, cv2.COLOR_RGB2BGR)  # Convert back to BGR for OpenCV saving

In [21]:
def sharpen_image_opencv(image):
    """Apply sharpening filter using OpenCV."""
    kernel = np.array([[0, -1, 0],
                       [-1,  5, -1],
                       [0, -1, 0]])
    sharpened = cv2.filter2D(image, -1, kernel)
    return sharpened

def sharpen_image_pil(image):
    """Apply unsharp mask using PIL for natural sharpening."""
    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    sharpened = pil_image.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3))
    return cv2.cvtColor(np.array(sharpened), cv2.COLOR_RGB2BGR)

In [18]:
def blend_frames(original, denoised, sharpened, alpha=0.5, beta=0.2):
    """
    Blend original, denoised, and sharpened frames using weighted averaging.
    
    Args:
        original (numpy.ndarray): Original frame (BGR).
        denoised (numpy.ndarray): Denoised frame (BGR).
        sharpened (numpy.ndarray): Sharpened frame (BGR).
        alpha (float): Weight for denoised frame.
        beta (float): Weight for sharpened frame.
        
    Returns:
        numpy.ndarray: Final blended frame.
    """
    # Ensure all images are the same shape
    assert original.shape == denoised.shape == sharpened.shape, "Frame size mismatch!"

    # Compute weighted blend
    blended = (alpha * denoised + beta * sharpened + (1 - alpha - beta) * original).astype(np.uint8)

    return blended

In [22]:
# def process_images(input_dir, output_dir, model, device):
#     """ Recursively denoise images while maintaining folder structure. """
#     input_path = Path(input_dir)
#     output_path = Path(output_dir)

#     for img_path in input_path.rglob('*'):
#         if img_path.suffix.lower() in ['.png', '.jpg', '.jpeg', '.bmp']:
#             relative_path = img_path.relative_to(input_path)
#             target_path = output_path / relative_path
#             target_path.parent.mkdir(parents=True, exist_ok=True)  # Ensure folder exists

#             # Apply denoising
#             denoised_img = denoise_image_swinir(model, img_path, device)
#             cv2.imwrite(str(target_path), denoised_img)  # Save output
#             print(f"Denoised and saved: {target_path}")

def process_images_with_sharpening(input_dir, output_dir, model, device, sharpening_method="opencv"):
    """Denoise images and apply sharpening while maintaining folder structure."""
    input_path = Path(input_dir)
    output_path = Path(output_dir)

    for img_path in input_path.rglob('*'):
        if img_path.suffix.lower() in ['.png', '.jpg', '.jpeg', '.bmp']:
            relative_path = img_path.relative_to(input_path)
            target_path = output_path / relative_path
            target_path.parent.mkdir(parents=True, exist_ok=True)  # Ensure folder exists

            # Apply denoising
            denoised_img = denoise_image_swinir(model, img_path, device)

            # Apply sharpening
            if sharpening_method == "opencv":
                final_img = sharpen_image_opencv(denoised_img)
            else:
                final_img = sharpen_image_pil(denoised_img)

            # Save final output
            cv2.imwrite(str(target_path), final_img)
            print(f"Denoised, sharpened, and saved: {target_path}")

def process_images_with_blending(input_dir, output_dir, model, device, alpha=0.45, beta=0.25):
    """
    Process images: Denoise, Sharpen, and Blend with the original.
    
    Args:
        input_dir (str): Folder with original extracted frames.
        output_dir (str): Folder to save final processed frames.
        model: SwinIR denoising model.
        device: Torch device (CPU or CUDA).
        alpha (float): Weight for denoised frame.
        beta (float): Weight for sharpened frame.
    """
    input_path = Path(input_dir)
    output_path = Path(output_dir)

    for img_path in input_path.rglob('*'):
        if img_path.suffix.lower() in ['.png', '.jpg', '.jpeg', '.bmp']:
            relative_path = img_path.relative_to(input_path)
            target_path = output_path / relative_path
            target_path.parent.mkdir(parents=True, exist_ok=True)  # Ensure folder exists

            # Load original frame
            original = cv2.imread(str(img_path), cv2.IMREAD_COLOR)

            # Apply denoising
            denoised = denoise_image_swinir(model, img_path, device)

            # Apply sharpening
            sharpened = sharpen_image_pil(denoised)  # or sharpen_image_opencv(denoised)

            # Blend frames
            final_img = blend_frames(original, denoised, sharpened, alpha, beta)

            # Save final output
            cv2.imwrite(str(target_path), final_img)
            print(f"Processed and saved: {target_path}")

In [23]:
# Ensure output directory exists
os.makedirs(TEST_SWINIR_FOLDER, exist_ok=True)

# Run batch processing
# process_images_with_sharpening(TEST_EXTRACTS_FOLDER, TEST_SWINIR_FOLDER, model, device, sharpening_method="opencv")
# process_images_with_sharpening(TEST_EXTRACTS_FOLDER, TEST_SWINIR_FOLDER, model, device, sharpening_method="pil")
process_images_with_blending(TEST_EXTRACTS_FOLDER, TEST_SWINIR_FOLDER, model, device, alpha=0.5, beta=0.2)

Processed and saved: D:\oldMemoriesTestArea\swinir\AVSEQ01_frames\frame_000000.png
Processed and saved: D:\oldMemoriesTestArea\swinir\AVSEQ01_frames\frame_000001.png
Processed and saved: D:\oldMemoriesTestArea\swinir\AVSEQ01_frames\frame_000002.png
Processed and saved: D:\oldMemoriesTestArea\swinir\AVSEQ01_frames\frame_000003.png
Processed and saved: D:\oldMemoriesTestArea\swinir\AVSEQ01_frames\frame_000004.png
Processed and saved: D:\oldMemoriesTestArea\swinir\AVSEQ01_frames\frame_000005.png
Processed and saved: D:\oldMemoriesTestArea\swinir\AVSEQ01_frames\frame_000006.png
Processed and saved: D:\oldMemoriesTestArea\swinir\AVSEQ01_frames\frame_000007.png
Processed and saved: D:\oldMemoriesTestArea\swinir\AVSEQ01_frames\frame_000008.png
Processed and saved: D:\oldMemoriesTestArea\swinir\AVSEQ01_frames\frame_000009.png
Processed and saved: D:\oldMemoriesTestArea\swinir\AVSEQ01_frames\frame_000010.png
Processed and saved: D:\oldMemoriesTestArea\swinir\AVSEQ01_frames\frame_000011.png
Proc

KeyboardInterrupt: 

## Upscaling

In [ ]:
def upscale_extracted_frames_limited(
    input_folder,
    output_folder,
    realesrgan_model_path,
    max_resolution=(1920, 1080),  # Limit max resolution to 1080p
    default_scale=4
):
    """
    Recursively upscale extracted frames in `input_folder` using Real-ESRGAN.
    Ensures that no frame exceeds `max_resolution` (e.g., 1920x1080).
    
    Args:
        input_folder (str): Directory containing frames to upscale.
        output_folder (str): Directory to save upscaled frames.
        realesrgan_model_path (str): Path to the Real-ESRGAN model file.
        max_resolution (tuple): Max (width, height) for upscaling.
        default_scale (int): Default upscale factor if resolution allows.
    """

    # 1) Define the RRDB model for Real-ESRGAN
    model = RRDBNet(
        num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=default_scale
    )

    # 2) Create RealESRGANer instance
    upsampler = RealESRGANer(
        scale=default_scale,
        model_path=realesrgan_model_path,
        model=model,
        tile=0,               # Set tile size if running out of memory
        tile_pad=10,
        pre_pad=0,
        half=False,           # Set to True for lower VRAM usage
        gpu_id=0
    )

    for root, _, files in os.walk(input_folder):
        for file_name in files:
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                input_path = os.path.join(root, file_name)
                rel_path = os.path.relpath(root, input_folder)
                target_subfolder = os.path.join(output_folder, rel_path)
                os.makedirs(target_subfolder, exist_ok=True)

                output_path = os.path.join(target_subfolder, file_name)
                img = cv2.imread(input_path, cv2.IMREAD_COLOR)
                
                if img is None:
                    print(f"Warning: Could not read {input_path}. Skipping.")
                    continue

                # Get original resolution
                orig_h, orig_w, _ = img.shape
                max_w, max_h = max_resolution

                # Determine the scaling factor to not exceed 1080p
                scale_w = max_w / orig_w
                scale_h = max_h / orig_h
                limited_scale = min(scale_w, scale_h, default_scale)

                # Ensure the scale is at least 1 (no downscaling)
                if limited_scale < 1:
                    print(f"Skipping {file_name}: Already above {max_resolution}.")
                    cv2.imwrite(output_path, img)  # Save original if already above 1080p
                    continue

                # Upscale while respecting the limit
                output, _ = upsampler.enhance(img, outscale=limited_scale)
                cv2.imwrite(output_path, output)
                print(f"Upscaled {file_name} with scale factor: {limited_scale:.2f}")

    print("All frames have been upscaled!")

# Create output directory if it doesn’t exist
os.makedirs(TEST_UPSCALED_FOLDER, exist_ok=True)

# Run the upscaling process with the 1080p limit
upscale_extracted_frames_limited(
    input_folder=TEST_SWINIR_FOLDER,
    output_folder=TEST_UPSCALED_FOLDER,
    realesrgan_model_path=REALESRGAN_MODEL_PATH,
    max_resolution=(1920, 1080),
    default_scale=4
)

c:\Users\Manan\anaconda3\envs\old_mems\Lib\site-packages\realesrgan\utils.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loadnet = torch.load(model_path, map_location=

All frames have been upscaled!


# ReStitching the video back together

In [25]:
# Ensure final output folder exists
os.makedirs(FINAL_VIDEO_FOLDER, exist_ok=True)

In [ ]:
def get_video_fps(video_path):
    """ Extract FPS from the original video using FFmpeg. """
    try:
        cmd = f'ffmpeg -i "{video_path}"'
        output = subprocess.run(cmd, shell=True, capture_output=True, text=True).stderr

        match = re.search(r'(\d+(?:\.\d+)?)\s*fps', output)
        if match:
            fps = float(match.group(1))
            print(f"✅ Extracted FPS: {fps}")
            return fps

    except subprocess.CalledProcessError as e:
        print(f"❌ FFmpeg Error:\n{e.stderr}")

    print("⚠ Warning: Could not detect FPS, using default 30 FPS.")
    return 30  # fallback

In [ ]:
def frames_to_video_with_audio(
    frames_folder,
    original_video_path,
    audio_file_path,
    final_video_path
):
    """
    Stitch frames in `frames_folder` into a video matching the FPS of `original_video_path`.
    If `audio_file_path` exists, merge audio into the final video.
    
    Args:
        frames_folder      (Path): Folder containing frame_000001.png etc.
        original_video_path (Path): Path to the original standardized .mp4 for FPS detection.
        audio_file_path    (Path): Path to the extracted AAC audio.
        final_video_path   (Path): Output .mp4 path for final stitched video.
    """
    # 1) Get FPS from the original standardized video
    fps = get_video_fps(original_video_path)

    # 2) Construct a temporary video path (no audio)
    temp_video_path = final_video_path.parent / f"{final_video_path.stem}_no_audio.mp4"
    temp_video_path.parent.mkdir(parents=True, exist_ok=True)

    # 3) Ensure frames exist
    png_files = list(frames_folder.glob("*.png"))
    if not png_files:
        print(f"❌ Frames folder is missing or empty: {frames_folder}")
        return

    # 4) Stitch frames into video
    # Using a zero-based index like frame_%06d.png.
    # Make sure your frames are indeed named like 'frame_000001.png', 'frame_000002.png', etc.
    cmd_frames = (
        f'ffmpeg -framerate {fps} '
        f'-i "{frames_folder / "frame_%06d.png"}" '
        f'-c:v libx264 -crf 18 -preset slow '
        f'"{temp_video_path}"'
    )
    print(f"Running FFmpeg command: {cmd_frames}")
    subprocess.run(cmd_frames, shell=True, check=True)

    # 5) Merge audio if available
    if audio_file_path.exists():
        cmd_audio = (
            f'ffmpeg -i "{temp_video_path}" '
            f'-i "{audio_file_path}" '
            f'-c:v copy -c:a aac -b:a 256k '
            f'"{final_video_path}"'
        )
        subprocess.run(cmd_audio, shell=True, check=True)

        # Remove temp video without audio
        temp_video_path.unlink(missing_ok=True)
        print(f"✅ Final video with audio saved: {final_video_path}")
    else:
        # No audio found → rename temp video to final
        temp_video_path.rename(final_video_path)
        print(f"⚠ No audio found, saved video without audio: {final_video_path}")


In [28]:
def restitch_all_videos(
    upscaled_folder,
    standardized_videos_folder,
    extracts_folder,
    final_videos_folder
):
    """
    Traverse `upscaled_folder` for all *frames directories, then restitch each into a final .mp4.
    
    Args:
        upscaled_folder (str/Path): Folder containing subfolders with upscaled frames ( .../<video_name>_frames ).
        standardized_videos_folder (str/Path): Folder containing the original standardized .mp4 files.
        extracts_folder (str/Path): Folder containing the extracted .aac audio files.
        final_videos_folder (str/Path): Destination folder for stitched final videos.
    """
    upscaled_path = Path(upscaled_folder)
    standard_path = Path(standardized_videos_folder)
    extracts_path = Path(extracts_folder)
    final_path = Path(final_videos_folder)

    for frames_dir in upscaled_path.rglob("*_frames"):
        if frames_dir.is_dir():
            # e.g. frames_dir = D:/oldMemoriesTestArea/upscaled/subfolder/myvideo_frames
            base_name = frames_dir.name.replace("_frames", "")  # "myvideo"
            
            # Figure out the relative path (excluding the final "_frames" portion)
            # e.g. relative_path = "subfolder"
            relative_path = frames_dir.relative_to(upscaled_path).parent

            # 1) Original standardized video path
            # e.g. D:/oldMemoriesTestArea/converted_videos/subfolder/myvideo.mp4
            original_video_path = standard_path / relative_path / f"{base_name}.mp4"

            # 2) Extracted audio path
            # e.g. D:/oldMemoriesTestArea/extracts/subfolder/myvideo.aac
            audio_file_path = extracts_path / relative_path / f"{base_name}.aac"

            # 3) Final output video path
            # e.g. D:/oldMemoriesTestArea/final_videos/subfolder/myvideo_final.mp4
            final_video_folder = final_path / relative_path
            final_video_folder.mkdir(parents=True, exist_ok=True)
            final_video_path = final_video_folder / f"{base_name}_final.mp4"

            # Debug logs
            print(f"=== Restitching: {frames_dir} ===")
            print(f"    -> Original video: {original_video_path}")
            print(f"    -> Audio file:     {audio_file_path}")
            print(f"    -> Output video:   {final_video_path}\n")

            # 4) Call the single-video restitch function
            frames_to_video_with_audio(
                frames_folder=frames_dir,
                original_video_path=original_video_path,
                audio_file_path=audio_file_path,
                final_video_path=final_video_path
            )


# Now just call this function with your paths
restitch_all_videos(
    upscaled_folder=TEST_UPSCALED_FOLDER,
    standardized_videos_folder=TEST_OUTPUT_FOLDER,
    extracts_folder=TEST_EXTRACTS_FOLDER,
    final_videos_folder=FINAL_VIDEO_FOLDER
)


=== Restitching: D:\oldMemoriesTestArea\upscaled\AVSEQ01_frames ===
    -> Original video: D:\oldMemoriesTestArea\converted_videos\AVSEQ01.mp4
    -> Audio file:     D:\oldMemoriesTestArea\extracts\AVSEQ01.aac
    -> Output video:   D:\oldMemoriesTestArea\final_videos\AVSEQ01_final.mp4

✅ Extracted FPS: 28.11
Running FFmpeg command: ffmpeg -framerate 28.11 -i "D:\oldMemoriesTestArea\upscaled\AVSEQ01_frames\frame_%06d.png" -c:v libx264 -crf 0 -preset slow "D:\oldMemoriesTestArea\final_videos\AVSEQ01_final_no_audio.mp4"
✅ Final video with audio saved: D:\oldMemoriesTestArea\final_videos\AVSEQ01_final.mp4
=== Restitching: D:\oldMemoriesTestArea\upscaled\flying car_frames ===
    -> Original video: D:\oldMemoriesTestArea\converted_videos\flying car.mp4
    -> Audio file:     D:\oldMemoriesTestArea\extracts\flying car.aac
    -> Output video:   D:\oldMemoriesTestArea\final_videos\flying car_final.mp4

✅ Extracted FPS: 10.03
Running FFmpeg command: ffmpeg -framerate 10.03 -i "D:\oldMemoriesT